In [ ]:
# %% === SCANNER DE ROMPIMENTOS (MÁXIMA / MÍNIMA + RSI) ===
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import pytz
from IPython.display import display

# ===============================================================
# 🧾 ÍNDICES DISPONÍVEIS (baseados na sua pasta):
# IBOV, IBRA, IBRX, IBXL, IC02, ICON, IDIV, IEEX, IFNC,
# IGCT, IGCX, IGNM, IMAT, IMOB, INDX, ISEE, ITAG, IVBX,
# MLCX, SMLL, UTIL
# ===============================================================

# 🧩 Configuração básica
BASE_PATH = r"C:\Users\flavi\Documents\PYTHON_FINANCAS\DATA_SCIENCE_PYTHON"
INDICE = "IBRA"  # exemplo: "IBOV", "IBRA", "SMLL", etc.

# ===============================================================
# ⚙️ CONFIGURAÇÕES GERAIS
# ===============================================================
CONFIG = {
    "timeframe": mt5.TIMEFRAME_D1,       # Ex: mt5.TIMEFRAME_H1, mt5.TIMEFRAME_H4, mt5.TIMEFRAME_D1
    "history_days": 300,                 # Histórico em dias
    "lookback": 200,                     # Janela usada para buscar máximas e mínimas
    "min_gap": 1,                        # Mínimo de candles entre rompimentos
    "rsi_period": 14,                    # Período do RSI
    "display_decimals": 2                # Casas decimais
}

# ===============================================================
# 🧠 FUNÇÕES AUXILIARES
# ===============================================================

def initialize_mt5():
    """Inicializa a conexão com o MetaTrader 5."""
    if not mt5.initialize():
        raise RuntimeError(f"Erro ao inicializar o MT5: {mt5.last_error()}")
    print("✅ Conexão com o MetaTrader 5 estabelecida.")

def calculate_rsi(df, window=14):
    """Calcula o RSI clássico."""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def get_mt5_data(ticker, timeframe, days):
    """Obtém dados históricos do MT5."""
    tz = pytz.timezone('America/Sao_Paulo')
    agora = datetime.now(tz)
    inicio = (agora - timedelta(days=days)).astimezone(pytz.utc)
    fim = agora.astimezone(pytz.utc)
    rates = mt5.copy_rates_range(ticker, timeframe, inicio, fim)
    if rates is None or len(rates) == 0:
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    # 🔹 Remove o último candle (ainda em formação)
    if len(df) > 0:
        df = df.iloc[:-1]
    return df

def find_breakouts(ticker, cfg):
    """Identifica rompimentos de máxima/mínima confirmados pelo fechamento."""
    df = get_mt5_data(ticker, cfg["timeframe"], cfg["history_days"])
    if df is None or len(df) < cfg["lookback"]:
        return None

    # Cálculo de máximas, mínimas e RSI
    df['max_lookback'] = df['high'].rolling(window=cfg["lookback"]).max()
    df['min_lookback'] = df['low'].rolling(window=cfg["lookback"]).min()
    df['RSI'] = calculate_rsi(df, window=cfg["rsi_period"])

    breakouts = []
    for i in range(cfg["lookback"], len(df)):
        high_now = df['high'].iloc[i]
        low_now = df['low'].iloc[i]
        close_now = df['close'].iloc[i]
        prev_max = df['max_lookback'].iloc[i - 1]
        prev_min = df['min_lookback'].iloc[i - 1]

        # ⚠️ Rompimento de mínima tem prioridade
        if low_now < prev_min and close_now < prev_min:
            idx_min = df['low'].iloc[i - cfg["lookback"]:i].idxmin()
            if (i - idx_min) >= cfg["min_gap"]:
                breakouts.append({
                    'Ticker': ticker,
                    'Data Rompimento': df['time'].iloc[i],
                    'Preço Rompimento': round(close_now, cfg["display_decimals"]),
                    'Tipo': 'Mínima',
                    'Valor Anterior': round(df['low'].iloc[idx_min], cfg["display_decimals"]),
                    'Data Anterior': df['time'].iloc[idx_min],
                    'RSI no Rompimento': round(df['RSI'].iloc[i], cfg["display_decimals"])
                })

        # ✅ Rompimento de máxima confirmado por fechamento
        elif high_now > prev_max and close_now > prev_max:
            idx_max = df['high'].iloc[i - cfg["lookback"]:i].idxmax()
            if (i - idx_max) >= cfg["min_gap"]:
                breakouts.append({
                    'Ticker': ticker,
                    'Data Rompimento': df['time'].iloc[i],
                    'Preço Rompimento': round(close_now, cfg["display_decimals"]),
                    'Tipo': 'Máxima',
                    'Valor Anterior': round(df['high'].iloc[idx_max], cfg["display_decimals"]),
                    'Data Anterior': df['time'].iloc[idx_max],
                    'RSI no Rompimento': round(df['RSI'].iloc[i], cfg["display_decimals"])
                })

    return breakouts if breakouts else None

# ===============================================================
# 🚀 EXECUÇÃO PRINCIPAL
# ===============================================================
if __name__ == "__main__":
    initialize_mt5()

    excel_path = fr"{BASE_PATH}\tickers_{INDICE}.xlsx"

    try:
        df_tickers = pd.read_excel(excel_path)
    except FileNotFoundError:
        mt5.shutdown()
        raise FileNotFoundError(f"❌ Arquivo não encontrado: {excel_path}")

    tickers = df_tickers['Ticker'].dropna().tolist()

    results = []
    for ticker in tickers:
        breakouts = find_breakouts(ticker, CONFIG)
        if breakouts:
            results.extend(breakouts)

    # Exibe resultados
    if results:
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(by='Data Rompimento', ascending=False)
        results_df = results_df.drop_duplicates(subset=['Ticker'], keep='first')

        timeframe_names = {
            mt5.TIMEFRAME_M15: "15 Minutos",
            mt5.TIMEFRAME_H1: "1 Hora",
            mt5.TIMEFRAME_H4: "4 Horas",
            mt5.TIMEFRAME_D1: "Diário"
        }
        tf_str = timeframe_names.get(CONFIG["timeframe"], str(CONFIG["timeframe"]))

        print(f"\n📊 Rompimentos detectados no índice {INDICE}:")
        print(f"Timeframe: {tf_str} | Janela: {CONFIG['lookback']} candles | RSI: {CONFIG['rsi_period']} períodos\n")
        pd.set_option('display.max_rows', None)
        display(results_df)
    else:
        print(f"⚠️ Nenhum rompimento relevante encontrado no índice {INDICE}.")

    mt5.shutdown()
